In [5]:
import requests
import json
import os
from dotenv import load_dotenv
# Groq API key
load_dotenv()

# Groq API key from environment variable
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
# === Helper Functions === #
def load_text(file_path):
    """Load text from a file."""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()
    except Exception as e:
        print(f"Error loading text file: {e}")
        return ""

def clean_text(text):
    """Clean the text by removing empty lines."""
    cleaned_text = "\n".join([line for line in text.splitlines() if line.strip() != ""])
    return cleaned_text

def query_groq_llm(prompt, context, max_tokens=500):
    """Send a prompt and context to the Groq LLM chat completion API."""
    try:
        url = "https://api.groq.com/openai/v1/chat/completions"
        headers = {
            "Authorization": f"Bearer {GROQ_API_KEY}",
            "Content-Type": "application/json"
        }
        payload = {
            "messages": [
                {"role": "system", "content": "You are an assistant helping summarize and query text."},
                {"role": "user", "content": f"{prompt}\n\nContext:\n{context}"}
            ],
            "model": "llama-3.3-70b-versatile",  # Replace with your desired model
            "max_tokens": max_tokens,  # Adjust based on your needs
            "temperature": 0.7  # Adjust randomness
        }
        response = requests.post(url, json=payload, headers=headers)
        if response.status_code == 200:
            return response.json().get("choices", [{}])[0].get("message", {}).get("content", "No response")
        else:
            print(f"Groq API Error: {response.status_code}, {response.text}")
            return f"An error occurred while fetching a response from Groq: {response.status_code}, {response.text}"
    except Exception as e:
        print(f"Error communicating with Groq API: {e}")
        return f"An error occurred while fetching a response from Groq: {e}"

# === Main Functionality === #
def process_text_file(file_path, output_txt_path):
    """Process the text file and generate a clean, compacted educational text file."""
    text = load_text(file_path)
    if not text:
        print("Failed to load text.")
        return

    # Clean the text
    cleaned_text = clean_text(text)

    # Use a generic query to summarize the entire content
    user_query = "Summarize the key points of the text."

    # Use Groq API for enhanced summarization
    print("\nQuerying Groq LLM for additional insights...")
    llm_response = query_groq_llm(user_query, cleaned_text, max_tokens=500)
    print("\nGroq LLM Response:")
    print(llm_response)

    # Save the summarized text to a file
    with open(output_txt_path, "w", encoding="utf-8") as f:
        f.write(llm_response)

# === Run the System === #
if __name__ == "__main__":
    file_path = "summary_blockchain.txt"  # Replace with your text file
    output_txt_path = "output.txt"
    process_text_file(file_path, output_txt_path)
    print(f"Summarized text saved to {output_txt_path}")


Querying Groq LLM for additional insights...

Groq LLM Response:
The text discusses the importance of cybersecurity and the benefits of using blockchain technology to prevent cyber attacks. Here are the key points summarized:

**Centralized vs Decentralized Security**

* Centralized networks are more vulnerable to cyber attacks and data breaches because all data is stored in one location.
* Decentralized networks use multiple nodes to store data, making it more secure and resilient.

**Benefits of Decentralization**

* Provides a trustless environment where no one has to know or trust anyone else.
* Improves data reconciliation by providing a shared view of data in real-time.
* Reduces points of weakness in systems by distributing resources and responsibilities.
* Optimizes resource distribution to provide better performance and consistency.

**Blockchain in Cyber Security**

* Blockchain technology can be used to secure online data by providing a decentralized, distributed ledger.
* 